<a href="https://colab.research.google.com/github/ratulbempu/ApneBoot-Clinical-Data-Analysis/blob/main/230301%20-%20AB%20Clinical%20Data%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

22-07-23 - I updated it so it loads logs and creates df_s and df_d faster. Also includes Log Source File. Works for Zug, need to check for nonin. 
Actual and Preset are slightly different because:
- Preset mode number shows up at the start of the apnea, Stim shows at start of stim +1 (instead of 5s it looks like 6 sec+1. This may be a rounding issue. Would be nice if they were on the same line but ok. 
- This rounding issue may also cause different apnea stimulation lengths by a second or two.


22-09-03 - 
- Added code to convert table to pdf

22-09-10 -
- Fix Smoothing Issue
- Add Thresholds 

22-19-18
Todo:
- X make individual plots ouput to pdf
- X - Good Enough - make summary table cleaner
  - reduce number of columns
  - increase overall font size
- X test with custom thresholds
- test and fix for nonin
- combine relevent pdfs into a report.
- test with Clinical Data

23-03-01
- want table with name, total time monitored, total time below X condition calculated), total stimulations, total stimulation time, total time under Y condition, total errorenous data. 
 



## **Initialize**

### Set Folder Name

### Import and Find Folders

In [2]:
#@title
import fileinput
import glob
import pandas as pd
import time
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import os
import linecache
from pathlib import Path
pd.options.mode.chained_assignment = None  # default='warn'
!pip install ipython-autotime
%load_ext autotime

%matplotlib inline
print ("Imports done")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.2 MB/s eta 0:00:00
Imports done
time: 2.47 ms (started: 2023-03-10 01:01:51 +00:00)


In [3]:
#@title
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive
time: 3min 9s (started: 2023-03-10 01:01:51 +00:00)


### Get Folders List

In [4]:
#@title
import os

def list_folders(startpath):
    list_path = []
    list_folder = []
    log = "log"
    LOG = "LOG"
    is_data_dir = 0;
    previous_name_path = ""
    for root, subdirs, files in os.walk(startpath):
        is_data_dir = 0
        for filename in files:
            current_name_path = os.path.join(root)
            if ((log in filename) | (LOG in filename)) & (previous_name_path != current_name_path):
                full_path = os.path.join(root,filename)
                list_folder.insert (0, os.path.basename (current_name_path))
                list_path.insert(0, root)
                previous_name_path = current_name_path

    return list_path, list_folder


time: 15.8 ms (started: 2023-03-10 01:05:01 +00:00)


### Import and Initialize Folders

## **Convert Files to Data Frames**

### Headers for Different Logger Versions

In [5]:
#@title

header = ""
settings_header = "" 
AB_Model = "default_string"
number_of_commas = 0


def decideDataLoggerVersion (df):
  # Read the third line's header and decide which data logger version is used
  # Assumes that the third line is unique for each data logger version
  # I need to check with Sourabh if the third line correlates with versions for the St. John's studies.
  global header
  global settings_header
  global AB_Model
  global number_of_commas
  
  raw_logfile_data_table_headers = ["default"]
  settings_table_headers = [""]
  data_table_headers = [""]
  settings_columns_rename_dict = [""]
  settings_columns_to_drop = [""]
  version = "unknown data code version"

  ZUG_DATA_HEADER = "dd/mm/yy,hh:mm:ss,SPO2,HR,PI,Status,Pulse-ox mode,Threshold Met,Stimulator Error,Stimulator ON/OFF,Power Mode,Battery voltage;"
  NONIN_DATA_HEADER = "dd/mm/yy,hh:mm:ss,SPO2,HR,Status,Pulse-ox mode,Threshold Met,Stimulator Error,Stimulator ON/OFF,Power Mode,Probe Off patient,Battery voltage"
  CLINICAL_SETTINGS_HEADER_INTERVENTION = "dd/mm/yy,hh:mm:ss,Intervention,Stimulator Intensity,Alarm intensity;"
  CLINICAL_SETTINGS_HEADER_PLACEBO = "dd/mm/yy,hh:mm:ss,Placebo,Stimulator Intensity,Alarm intensity;"

  settings_header = df.iat[0,0]
  header = df.iat[2,0]
  number_of_commas = header.count (',')
  print (settings_header)
  print (header)

  if header == ZUG_DATA_HEADER:
    version = "ZUG Circa April 2022"
    # print ("Detected Zug")
  elif (header == NONIN_DATA_HEADER):
      if (settings_header == CLINICAL_SETTINGS_HEADER_INTERVENTION):
          version = "Clinical M3-Intervention (Nonin) 2019"
          AB_Model = "M3-Intervention"
      elif (settings_header == CLINICAL_SETTINGS_HEADER_PLACEBO):
          version = "Clinical M3-Placebo (Nonin) 2019"
          AB_Model = "M3-Placebo"
      else:
          version = "NONIN Circa April 2022"
    # print ("Detected Nonin")
  elif number_of_commas == 5:
      version = "Clinical M1 (Nonin, Tuhin) 2019"
      AB_Model = "M1"
  elif number_of_commas == 9:
      version = "Clinical M2 (Nonin, Satish) 2019"
      AB_Model = "M2"
  else:
    print ("Not able to Detect Nonin Nor Zug")
    AB_Model = "Unable to detect AB Model"
  
  print (version)

  if version == "ZUG Circa April 2022":
    AB_Model = "ZUG Circa April 2022"
    raw_logfile_data_table_headers = [#"Settings Index",
                          #"Data Date-Time",
                           "Data Date",
             "Data Time",
            "SPO2",
            "HR",
            "PI",
            "Status",
            "Pulse-ox mode",
            "Threshold Met",
            "Stimulator Error",
            "Stimulator ON/OFF",
            "Power Mode",
        #     "Probe Off patient", - July 22, 2022 - RN checked this with Sourabh - this value was not being recorded although it is in the header
            "Battery voltage",
            #"Settings Index" # This is added at the end and is blank until filled
            ]
    data_table_headers = ["Settings Index",
                "Data Date-Time",
                "Data Date",
                "Data Time",
                "SPO2",
                "HR",
                "PI",
                "Status",
                "Pulse-ox mode",
                "Threshold Met",
                "Stimulator Error",
                "Stimulator ON/OFF",
               "Power Mode",   
            #   "Probe Off patient",
                "Battery voltage"
                ]
    settings_table_headers = [#'Settings Date-Time'
                              'Settings Index',
                                  'Settings Date',
                'Settings Time',
                'Preset mode',
                'SPO2 Threshold',
                "HR Threshold",
                "AND/OR",
                "Duration",
                "Stimulator Intensity",
                "Alarm intensity"]

    data_convert_dict = {
#                "Settings Index": int,
                "Data Date-Time": 'datetime64[ns]',
                "Data Date": 'datetime64[ns]',
                "Data Time": 'datetime64[ns]',
                "SPO2": int,
                "HR": int,
                "PI": float,
                "Status": int,
                "Pulse-ox mode":int,
                "Threshold Met":int,
                "Stimulator Error":int,
                "Stimulator ON/OFF":int,
                "Power Mode":int,
            #     "Probe Off patient":int,
                "Battery voltage": float
                #,"Settings Index": int
                }

    settings_convert_dict = {
#                "Settings Index": int,
                "Settings Date-Time": str,
                "Settings Date": str,
                'Settings Time':str,
                'Preset mode':int,
                'SPO2 Threshold':int,
                "HR Threshold":int,
                "AND/OR":int,
                "Duration":int,
                "Stimulator Intensity":int,
                "Alarm intensity":str}
    settings_columns_rename_dict = {
             "Data Date":"Settings Date",
             "Data Time": "Settings Time",
            "SPO2":"Preset mode",
            "HR":"SPO2 Threshold",
            "PI":"HR Threshold",
            "Status":"AND/OR",
            "Pulse-ox mode":"Duration",
            "Threshold Met":"Stimulator Intensity",
            "Stimulator Error":"Alarm intensity"
        #     "Stimulator ON/OFF",
        #     "Power Mode",
        # #     "Probe Off patient",
        #     "Battery voltage",
        #     "Settings Index" # This is added at the end and is blank until filled
        }
    
    settings_columns_to_drop = ["Stimulator ON/OFF",
           "Power Mode",
            "Battery voltage"]

  if version == "NONIN Circa April 2022":
    AB_Model = "NONIN Circa April 2022"
    raw_logfile_data_table_headers = [#"Settings Index",
                          #"Data Date-Time",
                           "Data Date",
             "Data Time",
            "SPO2",
            "HR",
         #   "PI",
            "Status",
            "Pulse-ox mode",
            "Threshold Met",
            "Stimulator Error",
            "Stimulator ON/OFF",
            "Power Mode",
        #     "Probe Off patient", - July 22, 2022 - RN checked this with Sourabh - this value was not being recorded although it is in the header
            "Battery voltage",
            "Extra Column"
            #"Settings Index" # This is added at the end and is blank until filled
            ]
    data_table_headers = ["Settings Index",
                "Data Date-Time",
                "Data Date",
                "Data Time",
                "SPO2",
                "HR",
          #      "PI",
                "Status",
                "Pulse-ox mode",
                "Threshold Met",
                "Stimulator Error",
                "Stimulator ON/OFF",
               "Power Mode",
            #   "Probe Off patient",
                "Battery voltage"
                ]
    settings_table_headers = [#'Settings Date-Time'
                              'Settings Index',
                                  'Settings Date',
                'Settings Time',
                'Preset mode',
                'SPO2 Threshold',
                "HR Threshold",
                "AND/OR",
                "Duration",
                "Stimulator Intensity",
                "Alarm intensity"]

    data_convert_dict = {
                #"Settings Index": int,
                # "Data Date-Time": 'datetime64[ns]',
                # "Data Date": 'datetime64[ns]',
                # "Data Time": 'datetime64[ns]',
                "SPO2": int,
                "HR": int,
           #     "PI": float,
                "Status": int,
                "Pulse-ox mode":int,
                "Threshold Met":int,
                "Stimulator Error":int,
                "Stimulator ON/OFF":int,
                "Power Mode":int,
            #     "Probe Off patient":int,
                "Battery voltage": float}
                #"Settings Index": int}

    settings_convert_dict = {
                # "Settings Date-Time": 'datetime64[ns]',
                # "Settings Date": 'datetime64[ns]',
                # 'Settings Time':'datetime64[ns]',
                'Preset mode':int,
                'SPO2 Threshold':int,
                "HR Threshold":int,
                "AND/OR":int,
                "Duration":int,
                "Stimulator Intensity":int,
                "Alarm intensity":str
                #"Settings Index": int
                }
    settings_columns_rename_dict = {

             "Data Date":"Settings Date",
             "Data Time": "Settings Time",
            "SPO2":"Preset mode",
            "HR":"SPO2 Threshold",
            "Status":"HR Threshold",
            "Pulse-ox mode":"AND/OR",
            "Threshold Met":"Duration",
            "Stimulator Error":"Stimulator Intensity",
            "Stimulator ON/OFF":"Alarm intensity"
        #     "Stimulator ON/OFF",
        #     "Power Mode",
        # #     "Probe Off patient",
        #     "Battery voltage",
        #     "Settings Index" # This is added at the end and is blank until filled
        }
    settings_columns_to_drop = [
           "Power Mode",
            "Battery voltage",
            "Extra Column"]

#   elif number_of_commas == 5:
#       version = "Nonin Clinical Study Model 2019"

  if version == "Clinical M1 (Nonin, Tuhin) 2019":

    raw_logfile_data_table_headers = [
                           "Data Date",
             "Data Time",
            "Status",
            "HR",
            "SPO2",
            "Stimulator ON/OFF",
            ]
    data_table_headers = ["Settings Index",
                "Data Date-Time",
                "Data Date",
                "Data Time",
                "Status",
                "HR",
                "SPO2",
          #      "PI",
                
                "Pulse-ox mode",
                "Threshold Met",
                "Stimulator Error",
                "Stimulator ON/OFF",
               "Power Mode",
            #   "Probe Off patient",
                "Battery voltage"
                ]
    settings_table_headers = [#'Settings Date-Time'
                              'Settings Index',
                                  'Settings Date',
                'Settings Time',
                'Preset mode',
                'SPO2 Threshold',
                "HR Threshold",
                "AND/OR",
                "Duration",
                "Stimulator Intensity",
                "Alarm intensity"]

    data_convert_dict = {
                #"Settings Index": int,
                # "Data Date-Time": 'datetime64[ns]',
                # "Data Date": 'datetime64[ns]',
                # "Data Time": 'datetime64[ns]',
                "SPO2": int,
                "HR": int,
           #     "PI": float,
                "Status": int,
            #     "Pulse-ox mode":int,
            #     "Threshold Met":int,
            #     "Stimulator Error":int,
                "Stimulator ON/OFF":int
            #     "Power Mode":int,
            # #     "Probe Off patient":int,
            #     "Battery voltage": float}
                #"Settings Index": int
                }

    settings_convert_dict = {
                # "Settings Date-Time": 'datetime64[ns]',
                # "Settings Date": 'datetime64[ns]',
                # 'Settings Time':'datetime64[ns]',
                # 'Preset mode':int,
                # 'SPO2 Threshold':int,
                # "HR Threshold":int,
                # "AND/OR":int,
                # "Duration":int,
                # "Stimulator Intensity":int
                # "Alarm intensity":str
                #"Settings Index": int
                }
    settings_columns_rename_dict = {

             "Data Date":"Settings Date",
             "Data Time": "Settings Time",
            # "Stimulator ON/OFF",
            # "Power Mode",
        #     "Probe Off patient",
            # "Battery voltage",
            # "Settings Index" # This is added at the end and is blank until filled
        }
    settings_columns_to_drop = [    
                    "Status",
            "HR",
            "SPO2",
            "Stimulator ON/OFF",        

            ]


  if version == "Clinical M2 (Nonin, Satish) 2019":
    raw_logfile_data_table_headers = [
                           "Data Date",
             "Data Time",
            "HR",
            "SPO2",
            "Status",
            "Unused Column 1",
            "Unused Column 2",
            "Unused Column 3",
            "Stimulator ON/OFF",
            "Unused Column 4",
             # <-- need to double check which column is stim off/on
            ]
    data_table_headers = ["Settings Index",
                "Data Date-Time",
                "Data Date",
                "Data Time",
                "HR",
                "SPO2",
          #      "PI",
                "Status",
                "Pulse-ox mode",
                "Threshold Met",
                "Stimulator Error",
                "Stimulator ON/OFF",
               "Power Mode",
            #   "Probe Off patient",
                "Battery voltage"
                ]
    settings_table_headers = [#'Settings Date-Time'
                              'Settings Index',
                                  'Settings Date',
                'Settings Time',
                'Preset mode',
                'SPO2 Threshold',
                "HR Threshold",
                "AND/OR",
                "Duration",
                "Stimulator Intensity",
                "Alarm intensity"]

    data_convert_dict = {
                #"Settings Index": int,
                # "Data Date-Time": 'datetime64[ns]',
                # "Data Date": 'datetime64[ns]',
                # "Data Time": 'datetime64[ns]',
                "SPO2": int,
                "HR": int,
           #     "PI": float,
                "Status": int,
            #     "Pulse-ox mode":int,
            #     "Threshold Met":int,
            #     "Stimulator Error":int,
                "Stimulator ON/OFF":int
            #     "Power Mode":int,
            # #     "Probe Off patient":int,
            #     "Battery voltage": float}
                #"Settings Index": int
                }

    settings_convert_dict = {
                # "Settings Date-Time": 'datetime64[ns]',
                # "Settings Date": 'datetime64[ns]',
                # 'Settings Time':'datetime64[ns]',
                # 'Preset mode':int,
                # 'SPO2 Threshold':int,
                # "HR Threshold":int,
                # "AND/OR":int,
                # "Duration":int,
                # "Stimulator Intensity":int
                # "Alarm intensity":str
                #"Settings Index": int
                }
    settings_columns_rename_dict = {

             "Data Date":"Settings Date",
             "Data Time": "Settings Time",
            # "Stimulator ON/OFF",
            # "Power Mode",
        #     "Probe Off patient",
            # "Battery voltage",
            # "Settings Index" # This is added at the end and is blank until filled
        }
    settings_columns_to_drop = [            
        "HR",	"SPO2",	"Status","Stimulator ON/OFF",
        "Unused Column 1",
            "Unused Column 2",
            "Unused Column 3",
            "Unused Column 4",
            ]

################ M3 ###########
  if ((version == "Clinical M3-Intervention (Nonin) 2019") | (version == "Clinical M3-Placebo (Nonin) 2019")):
  
    raw_logfile_data_table_headers = [#"Settings Index",
                          #"Data Date-Time",
                           "Data Date",
             "Data Time",
            "SPO2",
            "HR",
         #   "PI",
            "Status",
            "Pulse-ox mode",
            "Threshold Met",
            "Stimulator Error",
            "Stimulator ON/OFF",
            "Power Mode",
            "Probe Off patient", # clinical version has this column matching up, not sure if it is actually recording the correct data though
            "Battery voltage",
            ]
    data_table_headers = ["Settings Index",
                "Data Date-Time",
                "Data Date",
                "Data Time",
                "SPO2",
                "HR",
          #      "PI",
                "Status",
                "Pulse-ox mode",
                "Threshold Met",
                "Stimulator Error",
                "Stimulator ON/OFF",
               "Power Mode",
            #   "Probe Off patient",
                "Battery voltage"
                ]
    settings_table_headers = [#'Settings Date-Time'
                              'Settings Index',
                                  'Settings Date',
                'Settings Time',
                'Preset mode',
                'SPO2 Threshold',
                "HR Threshold",
                "AND/OR",
                "Duration",
                "Stimulator Intensity",
                "Alarm intensity"]

    data_convert_dict = {
                #"Settings Index": int,
                # "Data Date-Time": 'datetime64[ns]',
                # "Data Date": 'datetime64[ns]',
                # "Data Time": 'datetime64[ns]',
                "SPO2": int,
                "HR": int,
           #     "PI": float,
                "Status": int,
                "Pulse-ox mode":int,
                "Threshold Met":int,
                "Stimulator Error":int,
                "Stimulator ON/OFF":int,
                "Power Mode":int,
            #     "Probe Off patient":int,
                "Battery voltage": float}
                #"Settings Index": int}

    settings_convert_dict = {

                "Stimulator Intensity":int,
                "Alarm intensity":str
                #"Settings Index": int
                }
    ## Settings rename dictionary is different 
    settings_columns_rename_dict = {
             "Data Date":"Settings Date",
             "Data Time": "Settings Time",
            "SPO2":"Intervention or Placebo", 
            "HR" : "Stimulator Intensity",
            "Status":"Alarm intensity",
        }
    settings_columns_to_drop = [
            "Pulse-ox mode",
            "Threshold Met",
            "Stimulator Error",
            "Stimulator ON/OFF",
            "Power Mode",
            "Probe Off patient", # clinical version has this column matching up, not sure if it is actually recording the correct data though
            "Battery voltage",
]

#   elif number_of_commas == 5:
#       version = "Nonin Clinical Study Model 2019"


  return raw_logfile_data_table_headers, settings_table_headers, data_table_headers, settings_columns_rename_dict, settings_columns_to_drop, settings_convert_dict, data_convert_dict

time: 11.4 ms (started: 2023-03-10 01:05:01 +00:00)


### Compile & Split Log Files to df_d and df_s

In [6]:
#@title
# Reference here: https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe
file_list = []

def compileLogFiles (path):
  global file_list
#   print (path)
  os.chdir(path)
  file_list = sorted(glob.glob("*LOG*.csv")) + sorted(glob.glob("*log*.csv")) + sorted(glob.glob("*LOG*.txt")) + sorted(glob.glob("*log*.txt")) + sorted(glob.glob("*LOG*.TXT")) + sorted(glob.glob("*log*.TXT"))
              # in 2019 clinical, some files are log and some are LOG
#   file_list = sorted(glob.glob("*"))
  number_of_files = len(file_list)
#   print("Number of Files:" , len(file_list))
  print (file_list)
  li = []

  for file_name in file_list:
    # This could work faster potentially, but then the data logger identifier code for finding the zug/nonin code string has to be changed
    # df = pd.read_csv(file_name, header=None, sep='\n', names=list(range(12)))
  
    df = pd.read_csv(file_name, header=None, sep='\n')
    #df = pd.read_csv(file_name, header=None, sep=',')
    df['Source File'] = file_name
    li.append (df)
  
  frame = pd.concat(li, axis=0, ignore_index=True)
  return frame


time: 2.54 ms (started: 2023-03-10 01:05:01 +00:00)


In [7]:
#@title
def splitToDataAndSettingsDF (df_LOGs):
  
  raw_logfile_data_table_headers, settings_table_headers, data_table_headers, settings_columns_rename_dict, settings_columns_to_drop, settings_convert_dict, data_convert_dict = decideDataLoggerVersion (df_LOGs)    

  df_raw = df_LOGs[0].str.split(',', expand=True)
  #display (df_raw)
   
  df_raw.columns = raw_logfile_data_table_headers
  df_raw["Source File"] = df_LOGs ["Source File"]

  # Extract Settings
  df_raw_shift = df_raw.shift(periods=-1)
  df_s = df_raw_shift[(df_raw.iloc[:,2]=="Preset mode") | (df_raw.iloc[:,2]=="Intervention") | (df_raw.iloc[:,2]=="Placebo")]
  # For the clinical data - just use the rows with a new source file to keep the time/date and file. 
  if ((AB_Model == "M1") | (AB_Model == 'M2')):
    df_s = df_raw.drop_duplicates(subset=["Source File"], keep='first')


  # Delete settings and settings headers
  df_raw_shift = df_raw.shift(periods=1)                                        #shift down one row as a way to find one line below the settings header
  rows_below_header = df_raw[(df_raw_shift.iloc[:,2]=="Preset mode") |(df_raw_shift.iloc[:,2]=="Intervention")|(df_raw_shift.iloc[:,2]=="Placebo")]
  df_raw = df_raw.drop(rows_below_header.index)                                 #Drop all the setttings data lines 
  rows_with_header = df_raw[(df_raw.iloc[:,2]=="Preset mode") |(df_raw.iloc[:,2]=="Intervention")|(df_raw.iloc[:,2]=="Placebo")]
  df_raw = df_raw.drop (rows_with_header.index)                                 #Drops all the settings headers
  rows_with_header = df_raw[df_raw.iloc[:,2]=="SPO2"]
  df_raw = df_raw.drop (rows_with_header.index)                                 #Drops all the DATA headers
  
  # Format Settings Files
  df_s.drop(settings_columns_to_drop, axis=1, inplace=True)                     # Drop extra columns
  df_s.rename(columns=settings_columns_rename_dict, inplace=True)
  if 'Alarm Intensity' in df_s.columns:
    df_s['Alarm intensity'].apply(lambda v: v.rstrip(';'))
  
  #What's left of df_raw is now df_d for data
  df_d = df_raw
  if (AB_Model == "NONIN Circa April 2022"):
    df_d.drop (columns = "Extra Column", axis = 1, inplace=True)
    #df_s.drop (columns = "Extra Column", axis = 1, inplace=True)

  # In the clinical, Tuhin put P for the devices that were placebo and would have stimulated but actually didn't.
  # Replace those with the number 1
  df_d["Stimulator ON/OFF"].mask(df_d["Stimulator ON/OFF"] == " P", 1, inplace= True)
  

  
  
  if (AB_Model == "M1"):
    format_day = '%d-%MM-%yyyy'
    format_time = '%H:%m:%s'
  else:
    format_day = '%d/%m/%y'
    format_time = '%H:%M:%S'
    

#   if (AB_Model == "M1"):
#       try:
#         display (df_s)
#         display (df_d)
#         # df_s.head()
#         # df_d.head()
        
#         # print (df_d.columns)
#         # print ("df_s1 " + df_s["Settings Date"][3] + df_s["Settings Time"][3])
#         df_s.insert(loc=0, column="Settings Date-Time", value=pd.to_datetime(df_s["Settings Date"] + df_s["Settings Time"] ,format='%d-%MM-%yyyy %H:%m:%s'))
#         print ("df_d1 " + df_d["Data Date"][3] + df_d["Data Time"][3])
#         df_d.insert(loc=0, column="Data Date-Time", value=pd.to_datetime(df_d["Data Date"] + df_d["Data Time"] ,format='%d-%MM-%yyyy %H:%m:%s'))   
#         print ("matched 1!")
#       except Exception as e:
#           print (e)
#           print ("try alternate date format:")
#           display (df_s)
#         #   print (df_d.columns)
#           try:
#             # df_s.drop ("Settings Date-Time")
#             # df_d.drop ("Data Date-Time")
#             # df_s.insert(loc=0, column="Settings Date-Time", value=pd.to_datetime(df_s["Settings Date"] + df_s["Settings Time"] ,format='%d/%M/%yyyy %H:%m:%s'))
#             # df_d.insert(loc=0, column="Data Date-Time", value=pd.to_datetime(df_d["Data Date"] + df_d["Data Time"] ,format='%d/%M/%yyyy %H:%m:%s'))  
#             print ("matched 2!")
#           except Exception as e2:
#             print (e2)
#             print ("no more date formats to try!")       
 

#   else:
#     df_s.insert(loc=0, column="Settings Date-Time", value=pd.to_datetime(df_s["Settings Date"] + ' ' + df_s["Settings Time"] ,format='%d/%m/%y %H:%M:%S'))
#     df_d.insert(loc=0, column="Data Date-Time", value=pd.to_datetime(df_d["Data Date"] + ' ' + df_d["Data Time"] ,format='%d/%m/%y %H:%M:%S'))
#   print ("inserting with "+format_day + " " + format_time)
#   display (df_s)

#   print ("df_s insert done, here's what it looks like")
#   display (df_s)
#   display (df_d)

#   print ("df_d coerce done and now df_d looks like:")
#   display (df_d)
#   print ("inserts done")

# MARCH 9, 2023 - These two lines below work (infer_datetime_format=True needs to be checked bc some columns may have multiple date Time formats?) 
# but it's still slow so I'm commenting them out for now and replacing them with the two below

#   df_s.insert(loc=0, column="Settings Date-Time", value=pd.to_datetime(df_s["Settings Date"] + ' ' + df_s["Settings Time"], infer_datetime_format = True, errors = 'coerce'))
#   df_d.insert(loc=0, column="Data Date-Time", value=pd.to_datetime(df_d["Data Date"] + ' ' + df_d["Data Time"], infer_datetime_format = True, errors = 'coerce'))

  df_s.insert(loc=0, column="Settings Date-Time", value=df_s["Settings Date"] + ' ' + df_s["Settings Time"])
  df_d.insert(loc=0, column="Data Date-Time", value=df_d["Data Date"] + ' ' + df_d["Data Time"])



  if ((AB_Model == "M1") | (AB_Model == "M2") | (AB_Model == "M3-Placebo") | (AB_Model == "M3-Intervention")):
      df_s.insert(loc = 3, column="Preset mode", value = 1)
      df_s.insert(loc = 4, column="SPO2 Threshold", value = 0)
      df_s.insert(loc = 5, column="HR Threshold", value = 0)
      df_s.insert(loc = 6, column="AND/OR", value = 0)
      df_s.insert(loc = 7, column="Duration", value = 0)
      if ((AB_Model == "M1") | (AB_Model == "M2")):
        df_s.insert(loc = 8, column="Stimulator Intensity", value = 0)
        df_s.insert(loc = 9, column="Alarm intensity", value = 0)

  #Cleanup NaNs
  df_d = df_d.fillna(0)
  df_s = df_s.fillna(0)

  #Convert to best datatypes


#   df_s["Settings Date"]=pd.to_datetime(df_s["Settings Date"],format='%d/%m/%y')
#   df_s["Settings Time"]=pd.to_datetime(df_s["Settings Time"],format='%H:%M:%S')#.dt.time
#   df_d["Data Date"]=pd.to_datetime(df_d["Data Date"],format='%d/%m/%y')
#   df_d["Data Time"]=pd.to_datetime(df_d["Data Time"],format='%H:%M:%S')#.dt.time

#   df_s["Settings Date"]=pd.to_datetime(df_s["Settings Date"],format=format_day)
#   print ("Settings Date Updated done")
#   df_s["Settings Time"]=pd.to_datetime(df_s["Settings Time"],format=format_time)#.dt.time
#   print ("Settings Time Updated done")
  
#   df_d["Data Date"]=pd.to_datetime(df_d["Data Date"],format=format_day)
#   print ("Data Date Updated done")
  
#   df_d["Data Time"]=pd.to_datetime(df_d["Data Time"],format=format_time)#.dt.time
#   print ("Data Time Updated done")
  
## For Clinical Analysis I am not going to do these, since they are already done above when combined. 
#   df_s["Settings Date"]=pd.to_datetime(df_s["Settings Date"])
#   df_s["Settings Time"]=pd.to_datetime(df_s["Settings Time"], errors = 'coerce')#.dt.time
#   df_d["Data Date"]=pd.to_datetime(df_d["Data Date"])
#   df_d["Data Time"]=pd.to_datetime(df_d["Data Time"], errors = 'coerce')#.dt.time
  df_s.drop(columns = "Settings Date", axis = 1, inplace=True)
  df_s.drop(columns = "Settings Time", axis = 1, inplace=True)
  df_d.drop(columns = "Data Date", axis = 1, inplace=True)
  df_d.drop(columns = "Data Time", axis = 1, inplace=True)
  df_s=df_s.astype(settings_convert_dict, errors='raise')
  df_d=df_d.astype(data_convert_dict, errors='raise')

  #df_d = ResolveMidnightDateIssue (df_d) 

#  df_d["Data Date-Time"]=pd.to_datetime(df_d["Data Date-Time"],format='%d/%m/%y %H:%M:%S')

  # For Settings Table delete the confusing default numbers left in place when "preset" condition is on.
# 
  df_s = DeleteThresholdsForRowsUsingPresetConditions(df_s)
  df_s, df_d = addSettingsIndex (df_s,df_d)
  
  return df_s, df_d

time: 8.89 ms (started: 2023-03-10 01:05:01 +00:00)


time: 16.9 ms (started: 2023-03-10 01:05:01 +00:00)


In [8]:
# It was observed with Zug that for the first ~30 seconds after midnight, the date does not change. This needs to be fixed or "smoothed" with the code below.
#import calendar

#### This function is not finished.
def ResolveMidnightDateIssue (df_above):
  print ("in the smooth")
  df_diff = pd.DataFrame ()
  df_below = pd.DataFrame ()

  df_below["Data Date-Time"] = df_above["Data Date-Time"].shift (periods=1, fill_value=0)
  #df_below["Data Date"] = df_above["Data Date"].shift (periods=1, fill_value=0)
  df_below ["Date Diff"] = df_above["Data Date-Time"] < df_below["Data Date-Time"]
  df_below ["Date Diff Num"] = (df_above["Data Date-Time"] - df_below["Data Date-Time"]).dt.total_seconds()
  print (df_below ["Date Diff Num"].dtype)
  

  # display(df_above)
  # display(df_below)
  # display (df_below[df_below["Date Diff"]==1])
  df_midnight_bounds = df_below[df_below["Date Diff Num"].abs()>1000]
  #display (df_midnight_bounds)
  # for i in df_midnight_bounds.index:
  #   print (i)



  #display (df_below[df_below["Date Diff Num"]!=1])
  
  print ("Sum is: " + df_below["Date Diff"].astype(int).sum().astype(str))
  
  return df_above




time: 2.36 ms (started: 2023-03-10 01:05:01 +00:00)


In [9]:
#@title
def addSettingsIndex (df_s,df_d):
  #df_s["Settings Index"] = np.arange(len(df_s))
  df_s.insert(loc=0, column="Settings Index", value=np.arange(len(df_s))+1)
  df_d = pd.merge_asof (df_d,df_s["Settings Index"],left_index=True, right_index=True)
  SettingsIndexCol = df_d.pop('Settings Index')
  df_d.insert(0, 'Settings Index', SettingsIndexCol)
  
  return df_s, df_d

time: 1.82 ms (started: 2023-03-10 01:05:01 +00:00)


In [10]:
#@title
def DeleteThresholdsForRowsUsingPresetConditions (settings_table):
    # display(settings_table)
    for ind in settings_table.index:
        if settings_table["Preset mode"][ind] == 1:
            settings_table["SPO2 Threshold"][ind] = np.NaN
            settings_table["HR Threshold"][ind] = np.NaN
            settings_table["AND/OR"][ind] = np.NaN
            settings_table["Duration"][ind] = np.NaN
                       #Preset mode	SPO2 Threshold	HR Threshold	AND/OR	Duration
    return settings_table


time: 1.42 ms (started: 2023-03-10 01:05:01 +00:00)


## **Analyze Thresholds**

### Use Only Valid Data
This filters data based on device status. 

In [11]:
#@title
def GetIsValidColumn (df):
    is_valid = pd.DataFrame ()
    is_valid ["valid_vitals"] = ((df["SPO2"]>Data_Cleanup_SP02_LowCutOff) & (df["HR"]>Data_Cleanup_HR_LowCutOff) & (df["HR"] !=255) & (df["SPO2"] !=127))
    # print ("Num IsValid Vitals True:")
    # print (is_valid["valid_vitals"].sum())
    # print (is_valid ["valid_vitals"])
    # is_valid['valid_vitals'].value_counts()
    # print ("num valid vitals: " + is_valid [is_valid["valid_vitals"]==True].count)

    # Determine if status scale is 0-6 or 0-255...
    max_status = df["Status"].max()
    print ("about to check valid_status")
    display (df)
    # if (AB_Model == "M1"):
    if (max_status < 7):
        print ("eval status for 0-6 status")
        is_valid ["valid_status"] = (df['Status'] == 0);
    elif (max_status >= 7):
        print ("eval status for 0-255 status")
        is_valid ["valid_status"] = ((df['Status'] == 129) | (df['Status'] == 131));  
    
    # print ("Num IsValid Status True:")
    # print (is_valid["valid_status"].sum())
    
    is_valid['Is_Valid_Data'] = np.where((is_valid['valid_status'] == True) & (is_valid["valid_vitals"] == True), True, False)
    # is_valid ["Is_Valid_Data"] = ((is_valid['valid_vitals'] == True) & (is_valid ['valid_status'] == True))
    # is_valid['Is_Valid_Data'].value_counts()
    # print ("Num IsValid True:")
    # print (is_valid["Is_Valid_Data"].sum())
    return is_valid ["Is_Valid_Data"]




time: 2.71 ms (started: 2023-03-10 01:05:01 +00:00)


### Analyze Against Custom Thresholds
This function returns what is expected in the custom mode the user selected. It should match with actual stimulations in custom mode.

In [12]:
#@title
def AnalyzeWithCustomThresholds (df):
#   is_valid = ((df["SPO2"]>Data_Cleanup_SP02_LowCutOff) & (df["HR"]>Data_Cleanup_HR_LowCutOff))
#   is_valid = GetIsValidColumn (df)
  is_below_threshold_AND = is_valid & ((df["SPO2"]< df["SPO2 Threshold"]) & (df["HR"]< df["HR Threshold"]))
  is_below_threshold_OR = is_valid & ((df["SPO2"]< df["SPO2 Threshold"]) | (df["HR"]< df["HR Threshold"]))
  is_below_threshold_result = (df["AND/OR"]==0)*is_below_threshold_OR + (df["AND/OR"]==1)*is_below_threshold_AND
  # is_below_threshold_result = is_below_threshold_result.mask (is_below_threshold_result <= df["Duration"],0) #This statement removes any apnea durations that are below the threshold at that row where the apnea started.
  df_unmasked = calcDurationsCrossedThreshold(pd.DataFrame(is_below_threshold_result))

  df[Duration_Custom] = df_unmasked.mask(df_unmasked <= df["Duration"],0)


  return df

time: 1.8 ms (started: 2023-03-10 01:05:01 +00:00)


### Analyze With Preset Thresholds
This function is useful to see how the system would have performed if it were operating in preset mode.

In [13]:
#@title
SPO2_threshold_AND_condition = 85
HR_threshold_AND_condition = 100
SP02_threshold_OR_condition = 75
HR_threshold_OR_condition  = 90
Preset_Mode_Duration = 5

def AnalyzeWithPresetThresholds (df):
  
    
#   is_valid = ((df["SPO2"]>Data_Cleanup_SP02_LowCutOff) & (df["HR"]>Data_Cleanup_HR_LowCutOff))
#   is_valid = GetIsValidColumn (df)
  is_below_threshold_AND = is_valid & ((df["SPO2"] < SPO2_threshold_AND_condition) & (df["HR"]< HR_threshold_AND_condition))
  is_below_threshold_OR = is_valid & ((df["SPO2"]< SP02_threshold_OR_condition) | (df["HR"]< HR_threshold_OR_condition))

  # is_below_threshold_result = ((df["AND/OR"]==0))&(is_below_threshold_OR) | ((df["AND/OR"]==1) &is_below_threshold_AND)
  is_below_threshold_result = is_below_threshold_OR | is_below_threshold_AND
  #print ("Sum is:" + str(sum(is_below_threshold_result)))
#   display (is_below_threshold_result)
  # is_below_threshold_result = is_below_threshold_result.mask (is_below_threshold_result <= Preset_Mode_Duration,0) #This statement removes any apnea durations that are below the threshold at that row where the apnea started.
  # print ("Sum is:" + str(sum(is_below_threshold_result)))

  # print ("Total preset results are: " + str(is_below_threshold_result.sum()))
  df_unmasked = calcDurationsCrossedThreshold(pd.DataFrame(is_below_threshold_result))
  df[Duration_Preset] = df_unmasked.mask(df_unmasked <= Preset_Mode_Duration,0)

  #print ("Sum is:" + str(sum(df[Duration_Preset])))

  return df


time: 2.11 ms (started: 2023-03-10 01:05:01 +00:00)


### Analyze Actual Stimulations
This function looks at Stim Off/On field and counts the apnea durations (assuming SPO2 & HR are not 0). It does not consider the Preset/Custom settings at all. 

In [14]:
#@title
# Assumes it is being passed df_j
def AddNumActualStimulations (df):
  
#   Valid_Stimulations = df["Stimulator ON/OFF"]*(df["SPO2"]>Data_Cleanup_SP02_LowCutOff)*(df["HR"]>Data_Cleanup_HR_LowCutOff)
#   is_valid = GetIsValidColumn (df)
  Valid_Stimulations = df["Stimulator ON/OFF"]*is_valid
  df[Duration_Actual] = calcDurationsCrossedThreshold(pd.DataFrame(Valid_Stimulations))
   
  ## New code (22-07-23) - I later realized that with the new apnea duration counter, we don't need to "Stimulation started column" as it easily puts the apnea duration time where the stimulation started. 
  # df["Stimulator ON/OFF with Valid SPO2/HR"] = df["Stimulator ON/OFF"]*(df["SPO2"]!=0)*(df["HR"]!=0) 

  # df_previousrow = df.shift(periods=1)
  # isStimulationStarted = (
  #                         (df["Stimulator ON/OFF with Valid SPO2/HR"]==1) & 
  #                         (df_previousrow["Stimulator ON/OFF with Valid SPO2/HR"] == 0) &
  #                         (df["SPO2"]!=0) & 
  #                         (df["HR"]!=0)
  #                         )  # This condition was added because there are some odd cases where the stimulator turns when HR/SPO2 are 0...

  #This gives 408 rows, but the one above gives 411 rows. There are likely some times then the O2 or HR jumps to zero mid apnea but jumps back up and the stimulator is on the whole time? 
  # OldisStimulationStarted = (
  #                         (df["Stimulator ON/OFF"]==1) & 
  #                         (df_previousrow["Stimulator ON/OFF"] == 0) &
  #                         (df["SPO2"]!=0) & 
  #                         (df["HR"]!=0))

  #Should be 408 Rows
  
  # df["Stimulation Triggered (Actual Stimulator ON Triggered)"] = isStimulationStarted
  # df["Actual - Stimulation Durations (s)"] = calcDurationsCrossedThreshold(pd.DataFrame(df["Stimulator ON/OFF with Valid SPO2/HR"]))
  return df

time: 1.11 ms (started: 2023-03-10 01:05:01 +00:00)


### Threshold Cross Duration
This takes an array of "threshold crossed" or "stimulation on" (0,0,0,1,1,1,1,0..etc) and returns an array with the time in the first place the threshold is cross i.e. (0,0,0,4,0,0,0,0...)

In [15]:
#@title
# Input: A df with 0s and 1s representing cross thresholds
# see this -  https://stackoverflow.com/questions/62615666/how-to-sum-variable-ranges-in-a-pandas-column-to-another-column
def calcDurationsCrossedThreshold (df):
  
  df = df.iloc[::-1]
  a = df.iloc[:,0]==0
  blocks = a.cumsum()
  
  m = blocks.duplicated(keep='last')
  
  df['Sum'] = df.iloc[:,0].groupby(blocks).cumsum().mask(m)
  df = df.iloc[::-1]
  df_return = df ['Sum'].fillna(0)
  return df_return

time: 1.67 ms (started: 2023-03-10 01:05:01 +00:00)


## **Generate Summary Tables**

In [16]:
#@title
def reportApneaCountBySetting (df,df_s):
  table = pd.pivot_table (df, values = [Duration_Actual, Duration_Preset, Duration_Custom],
                          index = "Settings Index",
                          aggfunc='sum')
#   display (table)
  table = table.rename (columns = {Duration_Actual:SumDuration_Actual, Duration_Preset:SumDuration_Preset, Duration_Custom:SumDuration_Preset})
  df_s=df_s.join(table, on="Settings Index")
  

  #Count function counts zeros too, so need to replace zeros with NAs for this to work.
  df_nan = df.mask(df==0) # This will make all zeros into Nan
  table = pd.pivot_table (df_nan, values = [Duration_Actual, Duration_Preset, Duration_Custom],
                          index = "Settings Index",
                          aggfunc='count')
  
  table = table.rename (columns = {Duration_Actual:Count_Actual, Duration_Preset:Count_Preset, Duration_Custom:Count_Custom})
  df_s=df_s.join(table, on="Settings Index")
#   display (df_s)
  
  if (PRINT_TABLES == True):
    df_s.to_excel (folder_name + " - Summary of Events.xlsx")
    dataframe_to_pdf(df_s, folder_name + " - Apnea Count By Setting")
  return






time: 2.25 ms (started: 2023-03-10 01:05:01 +00:00)


In [17]:
#@title
def reportIndividualActualApneas (df):
  table_A = df[df[Duration_Actual]!=0]
  table_P = df[df[Duration_Preset]!=0]
  table_C = df[df[Duration_Custom]!=0]
  table_A_P = df[ (df["Preset mode"]==1 & (df[Duration_Actual]!=df[Duration_Preset]))]
  table_A_C = df[ (df["Preset mode"]==0 & (df[Duration_Actual]!=df[Duration_Custom]))]
                   
  if (PRINT_TABLES == True):
    with pd.ExcelWriter(folder_name + " - Individual Events.xlsx") as writer:
        table_A.to_excel (writer, sheet_name="Actual")
        table_P.to_excel (writer, sheet_name="Preset")
        table_C.to_excel (writer, sheet_name="Custom")
        table_A_P.to_excel (writer, sheet_name="Actual-Preset Discrepancy")
        table_A_C.to_excel (writer, sheet_name="Actual-Custom Discrepancey")

    dataframe_to_pdf(table_A, folder_name + " - Individual Events (Actual)")
  return



time: 2.54 ms (started: 2023-03-10 01:05:01 +00:00)


In [18]:
#@title
def convertDataframeToFigure (df):
  display (df)
  fig, ax =plt.subplots(figsize =(20,20))
  #fig, ax =plt.subplots(figsize=(len(df),len(df.columns)))
  ax.axis('tight')
  ax.axis('off')
  the_table = ax.table(cellText=df.values,colLabels=df.columns,loc='center')
  return fig




time: 1.41 ms (started: 2023-03-10 01:05:01 +00:00)


** ## Print Tables to PDF **
This code enables printing tables to pdf.

In [19]:
#@title
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import textwrap as twp

# #create column headers with long text and apply textwrap with \n after defined lenght
# columns=[twp.fill('this is a long column header text',25), twp.fill('...and this is an even longer column header text',25)]

# https://predictivehacks.com/?all-tips=save-a-pandas-dataframe-as-an-image
# pip install dataframe-image
# import pandas as pd
# import dataframe_image as dfi
 
# dfi.export(df, 'dataframe.png')

def _draw_as_table(df, pagesize):
    alternating_colors = [['white'] * len(df.columns), ['lightgray'] * len(df.columns)] * len(df)
    alternating_colors = alternating_colors[:len(df)]
    fig, ax = plt.subplots(figsize=pagesize)
    ax.axis('tight')
    ax.axis('off')
    # df=df.to_string(line_width=5)
    # .str.wrap(5)
    col_headers=list(df.columns.str.wrap(15)) #df.columns#.map("str")
    #display (col_headers)
    #print ("Cols is " + len(df[0]))
    #display (df)
    the_table = ax.table(cellText=df.values,
                        rowLabels=df.index,
                        colLabels=col_headers,
                        #colLabels=df.columns,
                        rowColours=['lightblue']*len(df),
                        colColours=['lightblue']*len(df.columns),
                        cellColours=alternating_colors,
                        loc='center')
    the_table.auto_set_column_width(col=list(range(len(df.columns)))) # Provide integer list of columns to adjust

    # the_table.auto_set_font_size(False)
    # the_table.set_fontsize(12)
    return fig
  

def dataframe_to_pdf(df, filename, numpages=(1, 1), pagesize=(11, 8.5)):
  with PdfPages(filename) as pdf:
    # display (df)
    # nh, nv = numpages
    # rows_per_page = len(df) // nh
    # cols_per_page = len(df.columns) // nv

    # nh, nv = numpages
    rows_per_page = 30
    cols_per_page = len(df.columns)
    nv = 1
    
    if len(df) % rows_per_page == 0: 
      nh = int(len(df) / rows_per_page)
    else:
      nh = int(len(df) / rows_per_page) +1
    
    # print ('num cols is ' + str(len(df.columns)))
    # print ('num rows is ' + str(len(df)) + "nv is " + str(nv))

    for i in range(0, nh):
        for j in range(0, nv):
            # print ("i is " + str(i) + "and j is " + str(j))
            page = df.iloc[(i*rows_per_page):min((i+1)*rows_per_page, len(df)),
                           (j*cols_per_page):min((j+1)*cols_per_page, len(df.columns))]
            # display (page)
            fig = _draw_as_table(page, pagesize)
            if nh > 1 or nv > 1:
                # Add a part/page number at bottom-center of page
                fig.text(0.5, 0.5/pagesize[0],
                         "Part-{}x{}: Page-{}".format(i+1, j+1, i*nv + j + 1),
                         ha='center', fontsize=8)
            
            pdf.savefig(fig, bbox_inches='tight')
            # plt.show ()
            plt.close()


time: 247 ms (started: 2023-03-10 01:05:01 +00:00)


## **Plots**

### Plot AB Data

In [20]:
#@title
def plotApneBootData (df,plot_title,x_axis_start,x_axis_end):
    #display (df)
    
    f, (ax, ax2) = plt.subplots(2, 1, gridspec_kw={'height_ratios': [10, 1]})

    ax2 = df.plot(x="Data Date-Time", y="Status", rot=0,ax=ax2)
    
    plotSPO2AndHR(df,plot_title,x_axis_start,x_axis_end,ax)
       
    f.tight_layout()
    
    return ax




time: 1.3 ms (started: 2023-03-10 01:05:02 +00:00)


In [21]:
#@title
def plotSPO2AndHR (df,plot_title,x_axis_start,x_axis_end,ax):
  df["Stimulator ON/OFF Plot"] = df["Stimulator ON/OFF"]*20




  ax = df.plot(x="Data Date-Time", y="SPO2", 
                            figsize=(30,20),
                            title = plot_title,
                            ylim=(-5,150),
                            xlim= (x_axis_start, x_axis_end),
                            grid=True,
                            #xticks=pd.date_range(x_axis_start, x_axis_end,24),
                            rot=0,
                            #kind='scatter',
                            color = 'red',
                            label = "SPO2",
                            linewidth=1, 
                            #s=2,
                            ax=ax)
    #ax.grid()
  df.plot(x="Data Date-Time", y="HR", 
                          #figsize=(20,5),
                          #title = current_date,
                          #ylim=(-5,150),
                          #xlim= (x_axis_start, x_axis_end),
                          #grid=True,
                          #xticks=pd.date_range(x_axis_start, x_axis_end,24),
                          #rot=1,
                          #kind='scatter',
                          color = 'blue',
                          label = "HR",
                          linewidth=1,
                          #s=2,
                          ax=ax)
  
  df.plot.line(x="Data Date-Time", y="Stimulator ON/OFF Plot", 
                          #figsize=(20,5),
                          #title = current_date,
                          #ylim=(-5,150),
                          #xlim= (x_axis_start, x_axis_end),
                          #grid=1 ,
                          #xticks=pd.date_range(x_axis_start, x_axis_end,24),
                          #rot=1,
                          #kind='scatter',
                          linewidth=4,
                          alpha=0.2,
               
                          #fmt ='+',
                          color = 'Orange',
                          label = "Stim ON",
                          #s=30,
                          ax=ax)
                          
  df_t = getThresholdTable (df)
  
  df_t.plot.line(x="Data Date-Time", y="SPO2 Threshold", 
                          #figsize=(20,5),
                          #title = current_date,
                          #ylim=(-5,150),
                          #xlim= (x_axis_start, x_axis_end),
                          #grid=1 ,
                          #xticks=pd.date_range(x_axis_start, x_axis_end,24),
                          #rot=1,
                          #kind='scatter',
                          color = 'DarkRed',
                          label = "SPO2 Threshold",
                          linewidth=3,
                          linestyle='dashed',
                          alpha=0.2,
                          #s=2,
                          ax=ax
                          )
  
  df_t.plot.line(x="Data Date-Time", y="HR Threshold", 
                          #figsize=(20,5),
                          #title = current_date,
                          #ylim=(-5,150),
                          #xlim= (x_axis_start, x_axis_end),
                          #grid=1 ,
                          #xticks=pd.date_range(x_axis_start, x_axis_end,24),
                          #rot=1,
                          #kind='scatter',
                          color = 'DarkBlue',
                          label = "HR Threshold",
                          linewidth=3,
                          linestyle='dotted',
                          alpha=0.2,
                          #s=2,
                          ax=ax
                          )
  
  x_offset = (x_axis_end - x_axis_start)/10
  df_a = df[(df[Duration_Actual] > 0)]
  #df_a["Text X Position"] = df_a["Data Date-Time"][ind] - x_offset
  df_a=df_a.reset_index(drop=True)
  df_a["Arrow Offet"] = df_a["Data Date-Time"] - x_offset

  for ind in df_a.index:
    if df_a["Arrow Offet"][ind] <= x_axis_start:
    #   print (str(df_a["Arrow Offet"][ind]) + "is less than " + str(x_axis_start))
      df_a["Arrow Offet"][ind] = df_a["Arrow Offet"][ind] + x_offset*2
      print ("so we replace with: " + str(df_a["Arrow Offet"][ind]))
    #print (ind)
    ax.annotate("Event For " + df_a[Duration_Actual][ind].astype(int).astype(str) + 's\n' + df_a["Data Date-Time"][ind].strftime('%Y-%m-%d %X') + '\n SPO2:' + df_a["SPO2"][ind].astype(str) + " HR:" + df_a["HR"][ind].astype(str),
            xy = (df_a["Data Date-Time"][ind], df_a["SPO2"][ind]), 
            fontsize = 10, 
            xytext = (df_a["Data Date-Time"][ind] - x_offset, ind%4*15+20), 
            arrowprops = dict(facecolor = 'yellow'),
            color = 'black',
            backgroundcolor='white')
    
  #df.plot.annotate(df[Duration_Actual].str(), xy="Data Date-Time", y=100, , fontsize = 10, color = 'g')

  #### PRINT THresholds
  
  # ax.plot.line (x=df["Data Date-Time"], y=df_t["HR Threshold"], 
  #                         #figsize=(20,5),
  #                         #title = current_date,
  #                         #ylim=(-5,150),
  #                         #xlim= (x_axis_start, x_axis_end),
  #                         #grid=1 ,
  #                         #xticks=pd.date_range(x_axis_start, x_axis_end,24),
  #                         #rot=1,
  #                         #kind='scatter',
  #                         color = 'Orange',
  #                         label = "HR Threshold",
  #                         linewidth=1,
  #                         #s=2,
  #                         ax=ax
  #                         )



  ax.grid('on', which='major')
  return ax

time: 4.89 ms (started: 2023-03-10 01:05:02 +00:00)


### Get Threshold Table

In [22]:
#@title
def getThresholdTable (df):
  SPO2_threshold_AND_condition = 85
  HR_threshold_AND_condition = 100
  SP02_threshold_OR_condition = 75
  HR_threshold_OR_condition  = 90
  Preset_Mode_Duration = 5

  df_t = pd.DataFrame()
  #df_t = df
  # df_t ["HR Threshold"] = 60
  # df_t ["SPO2 Threshold"] = 64
  df_t ["Data Date-Time"] = df["Data Date-Time"]
  df_t ["HR Threshold"] = df["Preset mode"]*HR_threshold_OR_condition
  df_t ["SPO2 Threshold"] = df["Preset mode"]*SP02_threshold_OR_condition
  df_t["HR Threshold"][df_t["HR Threshold"]==0]=df["HR Threshold"][df_t["HR Threshold"]==0]
  df_t["SPO2 Threshold"][df_t["SPO2 Threshold"]==0]=df["SPO2 Threshold"][df_t["SPO2 Threshold"]==0]

  # df_t ["HR Custom"] = df["HR Threshold"]
  # df["SPO2 Threshold"].notna().astype(int)*df["SPO2 Threshold"]
  # display (df_t)

  #is_below_threshold_AND = is_valid & ((df["SPO2"] < SPO2_threshold_AND_condition) & (df["HR"]< HR_threshold_AND_condition))
  
  #print ("df_t is: ")
  #display (df_t)

  return df_t

time: 1.23 ms (started: 2023-03-10 01:05:02 +00:00)


### Plot by Day

In [23]:
#@title
# df_plot=df_j

import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.dates as mdates


def plotByDay (df):
    locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
    formatter = mdates.ConciseDateFormatter(locator)

    df["Data Date"] = pd.to_datetime(df["Data Date"])
    date_list = np.unique(df["Data Date"])
    length = len(date_list)

    fname = folder_name + ' plot_by_day.pdf'
    with PdfPages(fname) as pdf:
        for i in range(length):
          current_date_64 = date_list[i]
          current_date=pd.to_datetime(current_date_64)
          
          # Create a dataframe of only one date
          df_current_date = df[(df["Data Date"] == current_date)]
          #display(df_current_date)
          
          # x axis start and end
          x_axis_start = current_date.replace(second = 1)
          #print(x_axis_start)
          x_axis_end = current_date.replace(hour = 23, minute=59, second = 59)
          
          
          ##### Sept 7 - Not sure if this below has to be commented or fixed
          # df_current_date["Data Time"] = pd.to_datetime(df_current_date["Data Time"].astype(str)).dt.time
          # #df_current_date["Data Time"] = pd.to_datetime(df_current_date["Data Time"])
          df_current_date["Data Time"] = df_current_date["Data Time"].dt.time
          df_current_date.set_index('Data Time')
          #display(df_current_date.dtypes)
          #display (df_current_date)
          
          plotApneBootData (df_current_date,current_date_64,x_axis_start,x_axis_end)
          pdf.savefig()
          plt.close()





time: 3.05 ms (started: 2023-03-10 01:05:02 +00:00)


### Plot By Hour

In [24]:
#@title
def plotByHour (df):
    locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
    formatter = mdates.ConciseDateFormatter(locator)

    df["Data Date"] = pd.to_datetime(df["Data Date"])
    df["Data Time"] = pd.to_datetime(df["Data Time"])
    df["Data Time-Hour"] = df["Data Date-Time"].dt.hour
    # display(df)
    # return
    date_list = np.unique(df["Data Date"])
    date_length = len(date_list)
    fname = folder_name + ' plot_by_hour.pdf'
    with PdfPages(fname) as pdf:

        firstPage = plt.figure(figsize=(11.69,8.27))
        firstPage.clf()
        txt = 'This is the title page'
        firstPage.text(0.5,0.5,txt, transform=firstPage.transFigure, size=24, ha="center")
        pdf.savefig()
        plt.close()




        for i in range(date_length):
          current_date_64 = date_list[i]
          current_date=pd.to_datetime(current_date_64)
          
          # Create a dataframe of only one date
          df_current_date = df[(df["Data Date"] == current_date)]
          #display(df_current_date)
      
          hour_list = np.unique(df_current_date["Data Time-Hour"])
          hour_length = len(hour_list)
          #print (hour_list)

          for j in range(hour_length):
              current_hour_64 = hour_list[j]
              #current_date=pd.to_datetime(current_date_64)
              df_current_hour = df_current_date[(df_current_date["Data Time-Hour"]==current_hour_64)]
              x_start = current_date + pd.offsets.Hour(current_hour_64)
              x_end = current_date + pd.offsets.Hour(current_hour_64+1)
              df_current_hour['Data Date-Time'].loc[df_current_hour.index[0]]
              # x_end = x_start + pd.offsets.Hour()
              
              # x_start = df_current_hour['Data Date-Time'].loc[df_current_hour.index[0]]
              # x_end = x_start + pd.offsets.Hour()
              title = str (current_date) + ' ' + str(x_start)
              plotApneBootData (df_current_hour,x_start,x_start,x_end)
              # plotApneBootData (df_current_hour,current_date,current_hour_64,current_hour_64+1)
              pdf.savefig()
              plt.close()


time: 2.85 ms (started: 2023-03-10 01:05:02 +00:00)


### SubPlot Grids of Apnea Events 

In [25]:
#@title
### Plot multiple subplots

def PlotApneaEventsGrid (df):
  
  start_offset = dt.timedelta (seconds=15)
  end_offset = dt.timedelta (seconds=45)
  num_rows = 2
  num_cols = 2
  num_subplots = num_rows * num_cols

  df_list_of_apnea_events = df[(df[Duration_Actual] != 0)]
  
  num_events = len(df_list_of_apnea_events)
  # print ("div " + str(num_events/num_subplots))
  # print ("div " + str(9/4))
  # print ("div " + str(np.ceil(9/4)))
  # #num_pages = int(num_events/num_subplots + 1)
  num_pages = int(np.ceil(num_events/num_subplots))
  #print ("number of pages is " + str (num_pages))
  #print ("number of events is " + str (num_events))
  current_page = 0
  current_event = 0
 
  fname = folder_name + ' plot_individual_events.pdf'
  with PdfPages(fname) as pdf:
    for page in range (num_pages):
      fig, axes = plt.subplots(nrows=num_rows, ncols=num_cols)
      # print (axes.flat)
      for ax in axes.flat:
        if (current_event < num_events):
          event_time = df_list_of_apnea_events.iloc[current_event].at["Data Date-Time"]
          plot_start = event_time - start_offset
          plot_end = event_time + end_offset
          df_event_data = df[((df["Data Date-Time"] > plot_start) & (df["Data Date-Time"] < plot_end))]
          #print ("Current Event Index is " + str(current_event))
          current_event = current_event + 1
          plot_title = folder_name + " Event " + str(current_event) + ":  " + str(event_time)
          plotSPO2AndHR (df_event_data,plot_title,plot_start,plot_end,ax)
    #   print ("tight layout")
      fig.tight_layout()
      pdf.savefig()
      plt.close()

time: 3.01 ms (started: 2023-03-10 01:05:02 +00:00)


## **Create Tables & Plots Functions**

### Column Names

In [26]:
#@title
# Global Variables
### Column Names
Duration_Actual = "Event Duration (s) [Actual Data]"
Duration_Custom = "Event Duration (s) [Using Custom Settings - Theoretical Calculation]"
Duration_Preset = "Event Duration (s) [Using Preset Settings - Theoretical Calculation]"

Count_Actual = "Event Count [Actual Data]"
Count_Custom = "Event Count [Using Custom Settings - Theoretical Calculation]"
Count_Preset = "Event Count (s) [Using Preset Settings - Theoretical Calculation]"

SumDuration_Actual = "Cumulative Duration (s) [Actual Data]"
SumDuration_Custom = "Cumulative Duration (s) [Using Custom Settings - Theoretical Calculation]"
SumDuration_Preset = "Cumulative Duration (s) [Using Preset Settings - Theoretical Calculation]"

time: 1.09 ms (started: 2023-03-10 01:05:02 +00:00)


### Create Data Tables

In [27]:
def createDataTables (data_path):
    df_LOGs = compileLogFiles (data_path)

    df_s, df_d = splitToDataAndSettingsDF (df_LOGs)
    # os.chdir(analysis_path)
    # df_d.to_csv (folder_name + " - Data.xlsx")

    # df_s.to_csv (folder_name + " - Settings.xlsx")
    # dataframe_to_pdf(df_s, folder_name + " - Settings")

    df_d["Is_Valid_Data"] = GetIsValidColumn(df_d);

    df_j = pd.merge (df_d,df_s, on='Settings Index')
    # df_j = AddNumActualStimulations(df_j)
    # df_j = AnalyzeWithCustomThresholds (df_j)
    # df_j = AnalyzeWithPresetThresholds (df_j)
    return df_j, df_d, df_s

time: 1.25 ms (started: 2023-03-10 01:05:02 +00:00)


### Create Plots

In [28]:
#@title
def createPlots (df_j):
    df_j.dtypes
    plotByDay (df_j)
    plotByHour (df_j)
    PlotApneaEventsGrid (df_j)

time: 810 µs (started: 2023-03-10 01:05:02 +00:00)


### Create Data Summary Tables

In [29]:
#@title
def createSummaryTables (df,df_s):
  
  df_s_columns_dropped = df_s.drop(['Settings Date','Settings Time','Alarm intensity'], axis=1)
  
  reportApneaCountBySetting (df,df_s_columns_dropped)
  reportIndividualActualApneas (df)

time: 955 µs (started: 2023-03-10 01:05:02 +00:00)


### Create Clinical Summary

In [30]:
#@title

def createClinicalSummaryTable (df_j, record_name):
    num_seconds_per_data_point = 1;
    if (AB_Model == "M1"):
        num_seconds_per_data_point = 2; # Tuhin's model M1 measured every 2 seconds. 
    # df_j["Is_Valid_Data"] = GetIsValidColumn (df_j)
    df_is_valid_data = df_j[df_j["Is_Valid_Data"] == True]
    df_clinical_summary = pd.DataFrame ()
    df_clinical_summary ["Name"] = [record_name]
    df_clinical_summary ["Time ApneBoot System Was On (hrs)"] = len(df_j.index) / 60 / 60 * num_seconds_per_data_point
    df_clinical_summary ["Time with valid vital and status readings (hrs)"] = df_j["Is_Valid_Data"].sum()/60/60 * num_seconds_per_data_point
    df_clinical_summary ["SPO2 - Avg"] = df_is_valid_data['SPO2'].mean()
    df_clinical_summary ["SPO2 - Std"] = df_is_valid_data['SPO2'].std()
    df_clinical_summary ["HR - Avg"] = df_is_valid_data['HR'].mean()
    df_clinical_summary ["HR - Std"] = df_is_valid_data['HR'].std()   
    df_clinical_summary ["Time SPO2 < 85 (hrs}"] = df_is_valid_data.query ('SPO2 < 85')['SPO2'].count() / 60 / 60 * num_seconds_per_data_point
    df_clinical_summary ["Time SPO2 < 80(hrs}"] = df_is_valid_data.query ('SPO2 < 80')['SPO2'].count() / 60 / 60 * num_seconds_per_data_point
    df_clinical_summary ["Time SPO2 < 75 (hrs}"] = df_is_valid_data.query ('SPO2 < 75')['SPO2'].count() / 60 / 60 * num_seconds_per_data_point
    df_clinical_summary ["Time SPO2 < 70 (hrs}"] = df_is_valid_data.query ('SPO2 < 70')['SPO2'].count() / 60 / 60 * num_seconds_per_data_point
    df_clinical_summary ["Time SPO2 < 65 (hrs}"] = df_is_valid_data.query ('SPO2 < 65')['SPO2'].count() / 60 / 60 * num_seconds_per_data_point
    return df_clinical_summary
#   df_s.insert(loc = 3, column="Preset mode", value = 1)

time: 2.07 ms (started: 2023-03-10 01:05:02 +00:00)


In [31]:
#@title
# df_j stands for df_joined
# display (df_j[df_j["Preset Threshold - Apnea Duration (s)"] !=0])
#df["Time (hours)"]={len(df)*1/60/60)
# df_j.to_csv (folder_name + " All_Data_Joined_Table.csv")
# df_s.to_csv (folder_name + " Settings_Index_Table.csv")

time: 426 µs (started: 2023-03-10 01:05:02 +00:00)


In [32]:
#@title
# display(df_j[df_j["Stimulation Triggered (Old Formula for Actual Settings)"]==1])
# display(df_j[df_j["Stimulation Triggered (New Formula for Actual Settings)"]==1])
# display(df_j[df_j["Stimulation Triggered (New Formula for Actual Settings)"]!= df_j["Stimulation Triggered (Old Formula for Actual Settings)"]])
# display(df_j[df_j["Preset Threshold - Apnea Duration (s)"]==0])

time: 386 µs (started: 2023-03-10 01:05:02 +00:00)


## **Main FOR LOOP Code**



In [33]:
#@title
# folder_name = "Geetha"
startpath = "/content/drive/MyDrive/Improve World Health/BEMPU Quality Management System Folder (Created by RN 211005)/3 - Product Specific Documents (TCF, etc)/BHPL-DHF – Design and Development/3)DHF-APNEBOOT_Rev 00/4. Design Verification _ Validation/DHF-01-28 - Design Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/"
# startpath = "/content/drive/MyDrive/Improve World Health/BEMPU Quality Management System Folder (Created by RN 211005)/3 - Product Specific Documents (TCF, etc)/BHPL-DHF – Design and Development/3)DHF-APNEBOOT_Rev 00/4. Design Verification _ Validation/DHF-01-28 - Design Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/St. Johns Phase 1 Data/intervention/Sneha"
# startpath = "/content/drive/MyDrive/Improve World Health/BEMPU Quality Management System Folder (Created by RN 211005)/3 - Product Specific Documents (TCF, etc)/BHPL-DHF – Design and Development/3)DHF-APNEBOOT_Rev 00/4. Design Verification _ Validation/DHF-01-28 - Design Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/St.Johns Phase 2 Data( >june 30,2020)/BO Hemabindu 2/BO Hemabindu 2 data"
# startpath = "/content/drive/MyDrive/Improve World Health/BEMPU Quality Management System Folder (Created by RN 211005)/3 - Product Specific Documents (TCF, etc)/BHPL-DHF – Design and Development/3)DHF-APNEBOOT_Rev 00/4. Design Verification _ Validation/DHF-01-28 - Design Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/St. Johns Phase 1 Data/"
import traceback

Data_Cleanup_SP02_LowCutOff = 50  #If the SPO2 value at the time of APnea is below 60, it must be a bad data point. 
Data_Cleanup_HR_LowCutOff = 50
AB_Model = " "

paths, folder_names = list_folders(startpath)
print (paths)
print (folder_names)

PRINT_TABLES = False
os.chdir(startpath)
os.makedirs("Analysis_Files", exist_ok=True)
analysis_path = startpath + "/Analysis_Files/"
os.chdir(analysis_path)
# errors = pd.DataFrame (columns = ['Folder Name',"Model",'Exception', "filename", "lineno", "funcname", "text"])
errors = pd.DataFrame ()
df_overall_clinical_summary = pd.DataFrame ()
df_overall_clinical_summary_detailed = pd.DataFrame ()

i = 0;
for path in paths:
    folder_name = folder_names[i]
    i += 1
    print ("on file " + str(i) + ' of ' + str(len(paths)))
    try:
        print ("Working on " + folder_name)
        os.chdir(analysis_path)
        os.makedirs("Analysis - " + folder_name, exist_ok=True)
        current_analysis_folder_path = analysis_path+folder_name +"/"
        os.chdir(current_analysis_folder_path)
        
        df_j, df_d, df_s = createDataTables (path)
        # print ("df_j - main")
        # display (df_j)
        # createPlots (df_j)
        # createSummaryTables (df_j,df_s)
        #####
        df_current_clinical_summary = pd.DataFrame ()
        df_current_clinical_summary = createClinicalSummaryTable (df_j, folder_name)
        df_current_clinical_summary["path"] = path[-100:]
        df_overall_clinical_summary = pd.concat([df_overall_clinical_summary, df_current_clinical_summary])
        os.chdir(analysis_path)
        df_overall_clinical_summary.to_excel ("Overall Clinical Summary.xlsx")
        
        #####
        df_current_clinical_summary_detailed = df_current_clinical_summary
        # df_current_clinical_summary_detailed["file_list"] = file_list
        df_current_clinical_summary_detailed["model"] = AB_Model;
        df_current_clinical_summary_detailed["data date-time"] = df_j["Data Date-Time"].iloc[0];
        df_overall_clinical_summary_detailed = pd.concat([df_overall_clinical_summary_detailed, df_current_clinical_summary_detailed])
        df_overall_clinical_summary_detailed.to_excel ("Overall Clinical Summary - Detailed.xlsx")
        print (folder_name + " success!")

    except Exception as exc: 
         print ("ERROR FOR " + folder_name + ":")
         filename, lineno, funcname, text = traceback.extract_tb(exc.__traceback__)[-1]
         print (exc)
         short_path = path[-100:]
         current_error = pd.DataFrame (data = {'short_path': [short_path], 'Folder Name':[folder_name], "Model": [AB_Model], 'Exception':[exc], "filename": [filename], "lineno": [lineno], "funcname": [funcname], "text": [text], 'data header': [header], 'settings header': [settings_header], 'numcommas':[number_of_commas]})
         errors = errors.append (current_error,ignore_index = True);
         os.chdir(analysis_path)
         errors.to_excel ("Error Summary.xlsx")


print ("errors table:")
display (errors)
        


    



['/content/drive/MyDrive/Improve World Health/BEMPU Quality Management System Folder (Created by RN 211005)/3 - Product Specific Documents (TCF, etc)/BHPL-DHF – Design and Development/3)DHF-APNEBOOT_Rev 00/4. Design Verification _ Validation/DHF-01-28 - Design Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/St. Johns Phase 1 Data/intervention/Baby of Parvati_intervention/Parvathi', '/content/drive/MyDrive/Improve World Health/BEMPU Quality Management System Folder (Created by RN 211005)/3 - Product Specific Documents (TCF, etc)/BHPL-DHF – Design and Development/3)DHF-APNEBOOT_Rev 00/4. Design Verification _ Validation/DHF-01-28 - Design Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/St. Johns Phase 1 Data/intervention/Johnsi', '/content/drive/MyDrive/Improve World Health/BEMPU Quality Management System Folder (Created by RN 211005)/3 - Product Specific Documents (TCF, etc)/BHPL-DHF – Design and Development/3)DHF-APNEBOOT_Rev 00/4. Design Ver

,Settings Index,Data Date-Time,Status,HR,SPO2,Stimulator ON/OFF,Source File
0,1,27-06-2019 22:39:32,163,255,127,0,27log29.csv
1,1,27-06-2019 22:39:34,167,255,127,0,27log29.csv
2,1,27-06-2019 22:39:36,163,255,127,0,27log29.csv
3,1,27-06-2019 22:39:38,167,255,127,0,27log29.csv
4,1,27-06-2019 22:39:40,135,255,80,0,27log29.csv
...,...,...,...,...,...,...,...
112074,1,30-06-2019 12:58:13,163,255,127,0,27log29.csv
112075,1,30-06-2019 12:58:15,227,255,127,0,27log29.csv
112076,1,30-06-2019 12:58:17,163,255,127,0,27log29.csv
112077,1,30-06-2019 12:58:19,167,255,127,0,27log29.csv


eval status for 0-255 status
Parvathi success!
on file 2 of 60
Working on Johnsi
['LOG00186.TXT']
16/07/19,08:47:45,000,000,3,0000,0,0,0,
16/07/19,08:47:47,150,095,0,0000,0,0,0,
Clinical M2 (Nonin, Satish) 2019
about to check valid_status


,Settings Index,Data Date-Time,HR,SPO2,Status,Unused Column 1,Unused Column 2,Unused Column 3,Stimulator ON/OFF,Unused Column 4,Source File
0,1,16/07/19 08:47:45,0,0,3,0000,0,0,0,,LOG00186.TXT
1,1,16/07/19 08:47:46,0,95,6,0000,0,0,0,,LOG00186.TXT
2,1,16/07/19 08:47:47,150,95,0,0000,0,0,0,,LOG00186.TXT
3,1,16/07/19 08:47:48,150,95,6,0000,0,0,0,,LOG00186.TXT
4,1,16/07/19 08:47:49,150,95,0,0000,0,0,0,,LOG00186.TXT
...,...,...,...,...,...,...,...,...,...,...,...
789176,1,25/07/19 12:00:39,173,98,0,0000,0,0,0,,LOG00186.TXT
789177,1,25/07/19 12:00:40,171,98,6,0000,0,0,0,,LOG00186.TXT
789178,1,25/07/19 12:00:41,171,98,0,0000,0,0,0,,LOG00186.TXT
789179,1,25/07/19 12:00:42,171,96,0,0000,0,0,0,,LOG00186.TXT


eval status for 0-6 status
Johnsi success!
on file 3 of 60
Working on surya
['10log22.csv', '12log5.csv']
10/6/2019 , 19:54:9, 171, 255, 127, 0
10/6/2019 , 19:54:13, 129, 167, 100, 0
Clinical M1 (Nonin, Tuhin) 2019
about to check valid_status


...

eval status for 0-6 status
Ahmedi success!
on file 60 of 60
Working on Apneboot data
['LOG01306.TXT']
30/12/19,13:47:37,000,000,3,0000,0,0,1,
30/12/19,13:47:39,000,000,3,0000,0,0,1,
Clinical M2 (Nonin, Satish) 2019
about to check valid_status


,Settings Index,Data Date-Time,HR,SPO2,Status,Unused Column 1,Unused Column 2,Unused Column 3,Stimulator ON/OFF,Unused Column 4,Source File
0,1,30/12/19 13:47:37,0,0,3,0000,0,0,1,,LOG01306.TXT
1,1,30/12/19 13:47:38,0,0,3,0000,0,0,1,,LOG01306.TXT
2,1,30/12/19 13:47:39,0,0,3,0000,0,0,1,,LOG01306.TXT
3,1,30/12/19 13:47:40,118,99,0,0000,0,0,1,,LOG01306.TXT
4,1,30/12/19 13:47:41,122,99,0,0000,0,0,1,,LOG01306.TXT
...,...,...,...,...,...,...,...,...,...,...,...
83329,1,31/12/19 12:56:24,137,97,6,0000,0,0,1,,LOG01306.TXT
83330,1,31/12/19 12:56:25,137,96,0,0000,0,0,1,,LOG01306.TXT
83331,1,31/12/19 12:56:26,137,95,0,0000,0,0,1,,LOG01306.TXT
83332,1,31/12/19 12:56:27,137,95,0,0000,0,0,1,,LOG01306.TXT


eval status for 0-6 status
Apneboot data success!
errors table:


,short_path,Folder Name,Model,Exception,filename,lineno,funcname,text,data header,settings header,numcommas
0,n/Clinical Validation/RCT ApneBoot/St.Johns Ph...,stj70221oct,M3-Placebo,'utf-8' codec can't decode byte 0xee in positi...,pandas/_libs/parsers.pyx,1917,pandas._libs.parsers.raise_parser_error,,"dd/mm/yy,hh:mm:ss,SPO2,HR,Status,Pulse-ox mode...","dd/mm/yy,hh:mm:ss,Placebo,Stimulator Intensity...",11
1,cal Validation/RCT ApneBoot/St.Johns Phase 2 D...,stj703_21oct,Unable to detect AB Model,local variable 'settings_convert_dict' referen...,<ipython-input-5-2b084a9f754e>,539,decideDataLoggerVersion,"return raw_logfile_data_table_headers, setting...","dd/mm/yy,hh:mm:ss,HR,SPO2,Status,Pulse-ox mode...","20/05/20,13:50:58,000,000,217,0,0,0,1,0,3.423",10
2,idation/Clinical Validation/RCT ApneBoot/St.Jo...,st701,Unable to detect AB Model,local variable 'settings_convert_dict' referen...,<ipython-input-5-2b084a9f754e>,539,decideDataLoggerVersion,"return raw_logfile_data_table_headers, setting...","20/05/20,13:40:13,000,000,217,0,0,0,1,0,3.450","20/05/20,13:40:09,000,000,217,1,0,0,1,0,3.450",10
3,ical Validation/RCT ApneBoot/St.Johns Phase 2 ...,stj70121oct,Unable to detect AB Model,local variable 'settings_convert_dict' referen...,<ipython-input-5-2b084a9f754e>,539,decideDataLoggerVersion,"return raw_logfile_data_table_headers, setting...","20/05/20,13:40:13,000,000,217,0,0,0,1,0,3.450","20/05/20,13:40:09,000,000,217,1,0,0,1,0,3.450",10
4,idation/Clinical Validation/RCT ApneBoot/St.Jo...,st702,Unable to detect AB Model,'utf-8' codec can't decode byte 0xee in positi...,pandas/_libs/parsers.pyx,1917,pandas._libs.parsers.raise_parser_error,,"20/05/20,13:40:13,000,000,217,0,0,0,1,0,3.450","20/05/20,13:40:09,000,000,217,1,0,0,1,0,3.450",10
5,/Clinical Validation/RCT ApneBoot/St.Johns Pha...,Stj03_01-sep,Unable to detect AB Model,local variable 'settings_convert_dict' referen...,<ipython-input-5-2b084a9f754e>,539,decideDataLoggerVersion,"return raw_logfile_data_table_headers, setting...","dd/mm/yy,hh:mm:ss,HR,SPO2,Status,Pulse-ox mode...","20/05/20,13:50:58,000,000,217,0,0,0,1,0,3.423",10
6,"ohns Phase 2 Data( >june 30,2020)/Extracted da...",STJ702_Intervention_Clinical data_24_July-2020...,Unable to detect AB Model,'utf-8' codec can't decode byte 0xee in positi...,pandas/_libs/parsers.pyx,1917,pandas._libs.parsers.raise_parser_error,,"dd/mm/yy,hh:mm:ss,HR,SPO2,Status,Pulse-ox mode...","20/05/20,13:50:58,000,000,217,0,0,0,1,0,3.423",10
7,"/St.Johns Phase 2 Data( >june 30,2020)/Extract...",STJ701_Placebo_Clinical data_24_July-2020_4PM,Unable to detect AB Model,local variable 'settings_convert_dict' referen...,<ipython-input-5-2b084a9f754e>,539,decideDataLoggerVersion,"return raw_logfile_data_table_headers, setting...","23/06/20,16:23:32,1,3,2;",NaN,4
8,Validation Report/Product Validation/Clinical ...,Apneboot data,M2,'utf-8' codec can't decode byte 0xcb in positi...,pandas/_libs/parsers.pyx,1917,pandas._libs.parsers.raise_parser_error,,"26/08/19,14:31:56,000,000,3,0000,0,0,0,","26/08/19,14:31:54,000,000,3,0000,0,0,0,",9
9,on Report/Product Validation/Clinical Validati...,Raw data of AB device,M2,'utf-8' codec can't decode byte 0xff in positi...,pandas/_libs/parsers.pyx,1917,pandas._libs.parsers.raise_parser_error,,"27/08/19,23:04:50,000,000,3,0000,0,0,0,","27/08/19,23:04:48,000,000,3,0000,0,0,0,",9


time: 11min 22s (started: 2023-03-10 01:05:02 +00:00)


In [40]:
### Match Table
startpath = "/content/drive/MyDrive/Improve World Health/BEMPU Quality Management System Folder (Created by RN 211005)/3 - Product Specific Documents (TCF, etc)/BHPL-DHF – Design and Development/3)DHF-APNEBOOT_Rev 00/4. Design Verification _ Validation/DHF-01-28 - Design Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/"
lookup_fn = "230310 AB RCT - Data Collection Summary RNv1 - Data Analysis Summary_Ratul.csv"
clinical_fn = "Overall Clinical Summary - Detailed.xlsx"

os.chdir(startpath)
df_lookup = pd.read_csv(lookup_fn)
os.chdir(analysis_path)
df_clinical = pd.read_excel(clinical_fn)

df_match = df_lookup.set_index('shortpath').join(df_clinical.set_index('path'))
columns_to_keep = [
# "Order",
"Unnamed: 1",
"Baby of",
"Start date of enrollment",
"End date of enrollment",
"Device_type",
"Number of days of enrollment",
"Birth weight (grams)",
"GA",
"Sub-Group",
# "Total hypoxia duration (in secs)",
# "Total hypoxia duration (in hh:mm:ss)",
"Ventilation",
# "86,400",
# "Time Enrolled (S)",
# "#Seconds in a day",
# "% seconds in Hypoxia",
# "Notes",
# "Notes 2",
# "Unnamed: 0",
# "Name",
"Time ApneBoot System Was On (hrs)",
"Time with valid vital and status readings (hrs)",
"SPO2 - Avg",
"SPO2 - Std",
"HR - Avg",
"HR - Std",
"Time SPO2 < 85 (hrs}",
"Time SPO2 < 80(hrs}",
"Time SPO2 < 75 (hrs}",
"Time SPO2 < 70 (hrs}",
"Time SPO2 < 65 (hrs}",
"model",
# "data date-time",
]

df_match = df_match [columns_to_keep]
display (df_match)
df_match.to_excel ("4 - Matched Table Summary.xlsx")



,Unnamed: 1,Baby of,Start date of enrollment,End date of enrollment,Device_type,Number of days of enrollment,Birth weight (grams),GA,Sub-Group,Ventilation,...,SPO2 - Avg,SPO2 - Std,HR - Avg,HR - Std,Time SPO2 < 85 (hrs},Time SPO2 < 80(hrs},Time SPO2 < 75 (hrs},Time SPO2 < 70 (hrs},Time SPO2 < 65 (hrs},model
"ApneBoot/St.Johns Phase 2 Data( >june 30,2020)/BO Chandrakala Twin 1/BO Chandrakala Twin 1 Raw Data",St. John's - Phase 2,Chandrakala Twin 1,30/06/2020,03/07/2020,Intervention,3,1660,30(+6),>28,No,...,95.030398,2.380263,141.611098,11.488341,0.392778,0.188889,0.126111,0.085278,0.041667,M3-Intervention
Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/Niloufer/Hafsa/AB Device data,Nilofer,Hafsa Sohail T-1,23/09/2019,01/10/2019,Placebo,8,850,26,<28,Yes,...,93.994115,3.959339,141.589597,15.339311,3.469167,1.253333,0.536389,0.159444,0.051111,M2
Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/Niloufer/Jyothi/Apneboot data,Nilofer,Jyothi,30/12/2019,01/01/2020,Placebo,2,1200,28,<28,No,...,97.665214,2.774600,139.725028,17.968092,0.035000,0.005556,0.000833,0.000833,0.000000,M2
Validation/Clinical Validation/RCT ApneBoot/St. Johns Phase 1 Data/placebo/Baby of Pavithra_placebo,St. John's - Phase 1,Pavitra,26/04/2019,04/05/2019,Placebo,8,1070,31(+3),>28,Yes,...,96.186925,2.697585,154.970736,7.543856,0.086667,0.020000,0.008333,0.003889,0.003333,M1
/Product Validation/Clinical Validation/RCT ApneBoot/St. Johns Phase 1 Data/intervention/Surya/surya,St. John's - Phase 1,Surya,07/06/2019,16/06/2019,Intervention,9,1020,32(+5),>28,Yes,...,93.040234,4.230694,166.936381,7.892070,3.427222,0.698333,0.090556,0.017778,0.002222,M1
"/Product Validation/Clinical Validation/RCT ApneBoot/St.Johns Phase 2 Data( >june 30,2020)/BO Anitha",St. John's - Phase 2,Anitha (B),06/10/2020,20/10/2020,Placebo,14,1700,28(+6),>28,Yes,...,90.654136,4.771940,158.852405,12.459632,20.503056,4.682500,1.525556,0.618056,0.192778,M3-Placebo
01-28 - Design Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/Niloufer/Ahmedi,Nilofer,Ahmedi T-2,06/11/2019,14/11/2019,Placebo,8,930,27,<28,No,...,96.953832,3.184831,155.400419,17.619141,1.536111,0.678611,0.141111,0.068333,0.038056,M2
1-28 - Design Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/Niloufer/Bhavani,Nilofer,Bhavani,29/08/2019,30/08/2019,Intervention,1,1250,28(+3),>28,Yes,...,94.479754,4.042257,136.086105,11.303564,1.059722,0.445833,0.127222,0.008333,0.000000,M2
1-28 - Design Validation Report/Product Validation/Clinical Validation/RCT ApneBoot/Niloufer/Tehseen,Nilofer,Tehseen Fathima,07/12/2019,12/12/2019,Placebo,5,1500,30,>28,Yes,...,96.157632,4.634523,131.283343,12.194401,0.895833,0.339722,0.198611,0.108056,0.056667,M2
"Product Validation/Clinical Validation/RCT ApneBoot/St.Johns Phase 2 Data( >june 30,2020)/BO Manjula",St. John's - Phase 2,Manjula,31/08/2020,19/09/2020,Placebo,19,930,29(+3),>28,Yes,...,94.582056,4.730680,158.716091,12.886100,12.628889,6.109444,3.571111,2.145278,1.292222,M3-Intervention


time: 236 ms (started: 2023-03-10 01:28:04 +00:00)


# DRAFT - New Section

## ** Not Used Code**

In [ ]:
#@title
# #fname = "Report.pdf"
# with PdfPages(fname) as pdf:
#   print("This is a test of the print function")
#   pdf.savefig()

In [ ]:
#@title
# usage_pv_data_table = pd.pivot_table(data_table, index =['Data Date',"Settings Index"],
#                          aggfunc = 'count')

# print(usage_pv_data_table)
# usage_pv_data_table.to_csv('Usage_pv_data_table.csv',index=True)
# usage_pv_data_table.to_excel("Usage_pv_data_table.xlsx")





In [ ]:
#@title
pd.reset_option("all")
#df = stimulation_results_table_with_all_thresholds
#print(df)
#pivot = pd.pivot_table(stimulation_results_table_with_all_thresholds,
                       #values=["Status"], 
#                       index=['Status'])
                       #aggfunc={'Status': np.count,
                       # 'HR': [min, max, np.mean]})

#df = df.Status.value_counts()
#print(pivot)
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', None)
# pd.set_option('display.max_colwidth', -1)
# pd.set_option("precision", 4)

#print(df)


#print(len (stimulation_results_table_with_all_thresholds))
#filtered_results_table_all = df[(df["Status"]==129) | (df["Status"]==131)]
#print(len (filtered_results_table_all))

#data_table = filtered_results_table_all

# pivot = pd.pivot_table(data_table,
#                        values=['SPO2','HR'], 
#                        index=['Data Date'],
#                        aggfunc={'SPO2': np.mean,
#                         'HR': [min, max, np.mean]})

#print(pivot)
#filtered_results_table_all.to_csv("Filtered_Results_Table_All.csv")

# 129    91112
# 153    49477
# 131    32220



Want 
- Date
- Setting Index
- 



In [ ]:
#@title
# # !sudo apt-get install wkhtmltopdf
# # !yum install wkhtmltopdf
# !apt-get install wkhtmltopdf
# import pandas as pd 
# df_apnea_summary_table.to_html('HTMLTest.html')  

# import pdfkit 

# pdfkit.from_url('HTMLTest.html', 'HTMLTest.pdf')

In [ ]:
#@title
df = pd.DataFrame({'Col': [1,4,5,6,7],
                   'Col2': [1,4,5,6,7],
                   })
display (df)
New_Col = "Col3"
df [New_Col] = df["Col"] +1

display (df)
display (df[New_Col])

In [ ]:
#@title
# Create a pandas dataframe with demo data:
#!pip install -e git+https://github.com/mindee/doctr.git#egg=python-doctr[torch]
#!pip install weasyprint
!pip install django-weasyprint
import pandas as pd
demodata_csv = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
df = pd.read_csv(demodata_csv)

def dfToPDF (df):
  # Pretty print the dataframe as an html table to a file
  intermediate_html = '/tmp/intermediate.html'
  to_html_pretty(df,intermediate_html,'Iris Data')
  # if you do not want pretty printing, just use pandas:
  # df.to_html(intermediate_html)

  # Convert the html file to a pdf file using weasyprint
  import weasyprint
  out_pdf= '/tmp/demo.pdf'
  weasyprint.HTML(intermediate_html).write_pdf(out_pdf)

  # This is the table pretty printer used above:

def to_html_pretty(df, filename='/tmp/out.html', title=''):
    '''
    Write an entire dataframe to an HTML file
    with nice formatting.
    Thanks to @stackoverflowuser2010 for the
    pretty printer see https://stackoverflow.com/a/47723330/362951
    '''
    ht = ''
    if title != '':
        ht += '<h2> %s </h2>\n' % title
    ht += df.to_html(classes='wide', escape=False)

    with open(filename, 'w') as f:
         f.write(HTML_TEMPLATE1 + ht + HTML_TEMPLATE2)

HTML_TEMPLATE1 = '''
<html>
<head>
<style>
  h2 {
    text-align: center;
    font-family: Helvetica, Arial, sans-serif;
  }
  table { 
    margin-left: auto;
    margin-right: auto;
  }
  table, th, td {
    border: 1px solid black;
    border-collapse: collapse;
  }
  th, td {
    padding: 5px;
    text-align: center;
    font-family: Helvetica, Arial, sans-serif;
    font-size: 90%;
  }
  table tbody tr:hover {
    background-color: #dddddd;
  }
  .wide {
    width: 90%; 
  }
</style>
</head>
<body>
'''

HTML_TEMPLATE2 = '''
</body>
</html>
'''

In [ ]:
#@title
# # PRINT TABLES & DATA
# from matplotlib.backends.backend_pdf import PdfPages
# #https://stackoverflow.com/questions/4042192/reduce-left-and-right-margins-in-matplotlib-plot
# #df_apnea_summary_table.plot()
# #dfToPDF (df)
# # fig = convertDataframeToFigure (apnea_summary_table)
# # pdf = PdfPages("foo.pdf")
# # pdf.savefig(fig, bbox_inches='tight')
# # pdf.close()
# # !pip install -c conda-forge python-pdfkit
# !pip3 install wkhtmltopdf
# !pip3 install pdfkit

# import pandas as pd
# import pdfkit as pdf
# import sqlite3

# # con=sqlite3.connect("baza.db")

# # df=pd.read_sql_query("select * from dobit", con)
# #df_apnea_summary_table.datetime = pd.to_datetime(apnea_summary_table.datetime)
# display(df_apnea_summary_table)

# df_apnea_summary_table.to_html('apnea_summary_table.html')
# nazivFajla='test.pdf'
# config = pdf.configuration(wkhtmltopdf=path_wkhtmltopdf)
# pdf.from_url('apnea_summary_table.html', nazivFajla, configuration=config)


